In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import numpy as np
import magic as magic
import scprep
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


sys.path.append(os.getcwd())

In [14]:
x = np.transpose(np.load('../SERGIO/imputation_data/DS6_clean_counts.npy'))
y = np.transpose(np.load('../SERGIO/imputation_data/DS6_45.npy'))
print(x.shape, y.shape)

(2700, 1200) (2700, 1200)


From SERGIO Paper: Prior to running MAGIC, we filtered the synthetic data for rare genes (those expressed in less
than 5 cells), and performed library size normalization as well as a square root transformation. We used MAGIC with the parameter t =
2, t = 7, and default setting where t is inferred from data

#### Filter for rare genes (expressed in < 5 cells)

In [15]:
#scprep.plot.plot_library_size(x, cutoff=5000, log=True)
y_hat = scprep.filter.filter_rare_genes(y, min_cells=5)
#scprep.plot.plot_library_size(x_hat, cutoff=5000, log=True)

#### Library size normalization and square root transformation

In [16]:
y_norm = scprep.normalize.library_size_normalize(y_hat)
y_norm = scprep.transform.sqrt(y_norm)
y_norm.shape

(2700, 1196)

#### Run MAGIC

In [17]:
def run_MAGIC(t_val):
    magic_op = magic.MAGIC(
        knn=5,
        knn_max=None,
        decay=1,
        # Variable changed in paper
        t=t_val,
        n_pca=100,
        solver="exact",
        knn_dist="euclidean",
        n_jobs=1,
        random_state=None,
        verbose=1,
    )

    y_hat = magic_op.fit_transform(y_norm, genes='all_genes')
    y_hat.shape

    save_str = 'yhat_MAGIC_t_' + str(t_val)
    np.save(save_str, y_hat)

In [19]:
for t in [2, 7, 'auto']:
    run_MAGIC(t)

Calculating MAGIC...
  Running MAGIC on 2700 cells and 1196 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 8.12 seconds.
    Calculating KNN search...
    Calculated KNN search in 0.57 seconds.
    Calculating affinities...
    Calculated affinities in 0.48 seconds.
  Calculated graph and diffusion operator in 9.18 seconds.
  Calculating imputation...
  Calculated imputation in 0.07 seconds.
Calculated MAGIC in 9.26 seconds.
